In [ ]:
!pip install datasketch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from datasketch import MinHash, MinHashLSH

In [ ]:
set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

In [ ]:
m1 = MinHash(num_perm=128) #liczba permutacji
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)

In [ ]:
for d in set1:
  m1.update(d.encode('utf8'))
for d in set2:
  m2.update(d.encode('utf8'))
for d in set3:
  m3.update(d.encode('utf8'))

In [ ]:
m1.jaccard(m2)

0.7109375

In [ ]:
m2.jaccard(m1)

0.7109375

In [ ]:
m1.jaccard(m3)

0.640625

In [ ]:
lsh = MinHashLSH(threshold=0.5, num_perm=128)

In [ ]:
lsh.insert('m2', m2)
lsh.insert('m3', m3)

In [ ]:
lsh.query(m1)

['m2', 'm3']

In [ ]:
import pickle
pickle.dump(lsh, open('plik.plk', 'wb'))

In [ ]:
lsh.b, lsh.r, lsh.h

(25, 5, 128)

In [ ]:
#Twenty years after by Alexandre Dumas and Auguste Maquet
!wget https://www.gutenberg.org/cache/epub/1259/pg1259.txt
#Hamlet, Prince of Denmark by William Shakespeare
!wget https://www.gutenberg.org/cache/epub/27761/pg27761.txt
#The Adventures of Sherlock Holmes by Arthur Conan Doyle
!wget https://www.gutenberg.org/cache/epub/1661/pg1661.txt
#The Picture of Dorian Gray by Oscar Wilde
!wget https://www.gutenberg.org/cache/epub/174/pg174.txt
#The Great Gatsby by F. Scott Fitzgerald
!wget https://www.gutenberg.org/cache/epub/64317/pg64317.txt

--2024-10-22 19:39:13--  https://www.gutenberg.org/cache/epub/1259/pg1259.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1473868 (1.4M) [text/plain]
Saving to: ‘pg1259.txt’

pg1259.txt          100%[===================>]   1.41M  5.61MB/s    in 0.3s    

2024-10-22 19:39:14 (5.61 MB/s) - ‘pg1259.txt’ saved [1473868/1473868]

--2024-10-22 19:39:14--  https://www.gutenberg.org/cache/epub/27761/pg27761.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228517 (223K) [text/plain]
Saving to: ‘pg27761.txt’

pg27761.txt         100%[===================>] 223.16K  --.-KB/s    in 0.1s    

2024-10-22 19:39:14 (1.52 

Możemy stworzyć folder books i przenieść tam książki manualnie

In [ ]:
#tworzymy folder books i wrzucamy tam książki (cp kopiuje z folderu books do folderu głównego)
#!mkdir books
#!mv pg1259.txt pg1661.txt pg174.txt pg27761.txt books/
#!cp ./books/pg1259.txt ./

In [ ]:
import time
import os
import re

In [ ]:
stopwords = {"ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they",
             "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or",
             "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor",
             "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had",
             "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then",
             "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too",
             "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it",
             "how", "further", "was", "here", "than"}

In [ ]:
def read_words(f):
  return [word for line in open(f, 'r') for word in line.split()]

In [ ]:
def shinglets(f, N=5):
  shinglet = []
  with open(f, 'r') as f_:
    for line in f_:
      line = re.sub(r'[^\w]', '', line)
      line = line.replace(' ', '')
      line = line.lower()
      line = line.split()
      for w in line:
        if w in stopwords:
          line.remove(w)
      for s in range(len(line) - N):
        shinglet.append(line[s:s+N])
  return shinglet

## LSH v1 z datasketch

In [ ]:
from os.path import isfile, join

In [ ]:
def LSHcount(dirpath, filepath, threshold=0.7):
  files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]

  start = time.time()
  print("Zaczynam tworzenie MinHasha")

  lsh = MinHashLSH(threshold=threshold, num_perm=128)
  for f in files:
    doc = shinglets(join(dirpath, f))
    mhash = MinHash()
    for shinglet in doc:
      mhash.update(shinglet.encode('utf8'))
    lsh.insert(f, mhash)

  end_1 = time.time()
  print("Kończę tworzenie MinHasha. Czas: ", end_1 - start)

  search_doc = shinglets(filepath)
  m = MinHash()
  for shinglet in doc:
    m.update(shinglet.encode('utf8'))
  result = lsh.query(m)

  end_2 = time.time()
  print("Wykonanie query. Czas: ", end_2 - end_1)

  print(result)

In [ ]:
#jako drugi argument podajemy książkę której nie wrzuciliśmy do folderu books
LSHcount('./books/', 'pg1259.txt')

Zaczynam tworzenie MinHasha
Kończę tworzenie MinHasha. Czas:  0.3667285442352295
Wykonanie query. Czas:  0.20117902755737305
['pg174.txt', 'pg1259.txt', 'pg27761.txt', 'pg1661.txt']


In [ ]:
#napisać swojego jaccarda i porównać
def Jaccard(A, B):
  TODO